In [45]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv(
    "../.env"
)
client = OpenAI(api_key=os.getenv("OPENAI_KEY"))

## Creo un Vectorstore
Per prima cosa, si crea un vectorstore vuoto. Basta specificare un nome.  
Una volta ultimata la creazione del vectorstore vuoto, la variabile con cui viene istanziato conterrà (tra gli altri) l'attributo "id".  
Questo id sarà il riferimento unico del nostro vectorstore.


In [46]:
vs_name = "Vectorstore_prova_notebook"
vector_store = client.beta.vector_stores.create(name=vs_name)

vector_store.id

'vs_emFudN5h5cw92LnDnaNSf7Qv'

## Aggiunta di un file al VectorStore

Questo è il modo in cui si aggiunge un file ad un vectorstore. Nota: deve essere uno stream, come testimonia l'uso di open.  
Utilizzo il file stream per caricare il file su OpenAI. L'upload del file su OpenAI (comando client.files.create), una volta concluso, popola l'oggetto istanziato con l'attributo file.id.  

A questo punto possiamo utilizzare il file.id appena creato e usare il comando client.beta.vector_stores.file.create per caricare l'id di un file sull'id di un vectorstore.  

Per questa prova utilizzo il vector_store.id delle celle passate.

In [47]:
local_path = r"C:\Users\moglial\Downloads\OpenAI's Trust Center documentation\OpenAI API_ChatGPT 2023 SOC 2 Type 2 Report.pdf"
with open(local_path, "rb") as file_stream:
            file = client.files.create(
                file=file_stream,
                purpose="assistants"
                )
# aggiungo il file al vectorstore
client.beta.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file.id
)

VectorStoreFile(id='file-XTTie8G2pSLpFe5rm2zZwEKo', created_at=1717577221, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_emFudN5h5cw92LnDnaNSf7Qv', chunking_strategy={'type': 'static', 'static': {'max_chunk_size_tokens': 800, 'chunk_overlap_tokens': 400}})

## Stampiamo una lista dei vectorstores esistenti.

Possiamo stampare una lista dei vs presenti sul progetto che stiamo utilizzando. Il progetto è determinato dalla chiave api che stiamo utilizzanfo quando istanziamo il client di openai

In [48]:
vector_stores = client.beta.vector_stores.list()
print(vector_stores)

SyncCursorPage[VectorStore](data=[VectorStore(id='vs_emFudN5h5cw92LnDnaNSf7Qv', created_at=1717577219, file_counts=FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1), last_active_at=1717577221, metadata={}, name='Vectorstore_prova_notebook', object='vector_store', status='completed', usage_bytes=216776, expires_after=None, expires_at=None), VectorStore(id='vs_9Fss20NPVQgCdN2vhKR69Obp', created_at=1717513095, file_counts=FileCounts(cancelled=0, completed=4, failed=0, in_progress=0, total=4), last_active_at=1717516660, metadata={}, name='sicurezza informatica', object='vector_store', status='completed', usage_bytes=1920643, expires_after=None, expires_at=None), VectorStore(id='vs_ximj63NaqSEfYdtrgNOztj3S', created_at=1717505253, file_counts=FileCounts(cancelled=0, completed=9, failed=0, in_progress=0, total=9), last_active_at=1717512217, metadata={}, name='DOCUMENTAZIONE OPENAI', object='vector_store', status='completed', usage_bytes=704132, expires_after=None, expire

Anche i vectorsotres sono racchiusi in un oggetto SyncCursorPage. Questo oggetto è un po' menoso da utilizzare, ma è un iterabile. Questo significa che possiamo usarlo per un ciclo for, oppure (come nel nostro caso) per trasformarlo in una lista utilizzando la hotkey list().

Osserviamo che ciasun oggetto VectorStore ha un id, il timestamp in secondi di quando è stato creato, il filecount, l'ultimo accesso, eventuali metadati che possiamo definire noi, e il nome. Sono anche presenti alcuni parametri di policy come usage_bytes , expires_after (auto-eliminazione dei files dopo un certo periodo di non utilizzo), expires_at (per l'eliminazione schedulata di un VS)

Ricordiamoci che lo storage dei vectorstores costa 0.10$ al giorno per GB per i GB dopo il primo che è gratis. 

In [49]:
vector_stores_list = list(vector_stores)
vector_stores_list

[VectorStore(id='vs_emFudN5h5cw92LnDnaNSf7Qv', created_at=1717577219, file_counts=FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1), last_active_at=1717577221, metadata={}, name='Vectorstore_prova_notebook', object='vector_store', status='completed', usage_bytes=216776, expires_after=None, expires_at=None),
 VectorStore(id='vs_9Fss20NPVQgCdN2vhKR69Obp', created_at=1717513095, file_counts=FileCounts(cancelled=0, completed=4, failed=0, in_progress=0, total=4), last_active_at=1717516660, metadata={}, name='sicurezza informatica', object='vector_store', status='completed', usage_bytes=1920643, expires_after=None, expires_at=None),
 VectorStore(id='vs_ximj63NaqSEfYdtrgNOztj3S', created_at=1717505253, file_counts=FileCounts(cancelled=0, completed=9, failed=0, in_progress=0, total=9), last_active_at=1717512217, metadata={}, name='DOCUMENTAZIONE OPENAI', object='vector_store', status='completed', usage_bytes=704132, expires_after=None, expires_at=None),
 VectorStore(id='vs

## Creazione di un assistant

Un assistant è l'entità che si occuperà di generare le risposte sulla base della documentazione fornita nel vectorstore.  
Quando creiamo un assistente, istanziamo un oggetto assistant con 

```python
assistente = client.beta.assistants.create(
            instructions = # Il prompt dell'assistente. Nel nostro caso potrebbe essere qualcosa come "Sei una AI che deve rispondere a delle domande su della documentazione in maniera esaustiva e professionale",  
            name = # Il nome che vogliamo dare all'assistant. Questo ci servirà per avere un livello di identificazione più umano rispetto al solo assisitente.id,  
            tools = # Qua dobbiamo mettere una lista dei tools di cui vogliamo dotare l'assistant. Per ora ne sono previsti 3 : file_search (quello che utilizzeremo), code_interpreter e function_calling. La sintassi è: [{"type":"file_search"}, {"type":"code_interpreter"}, ... ]  
            model = # Il modello che vogliamo avere come motore dell'assistant. Consiglio di utilizzare gpt-3.5-turbo (veloce e economico) oppure gpt-4o (più economico di gpt-4, ma più veloce)  
)

In [50]:
new_assistant = client.beta.assistants.create(
                instructions="Sei un helper aziendale e devi aiutare a rispondere a delle domande basate sulla documentazione che ti è stata fornita.",
                name="Assistente di prova",
                tools=[{"type": "file_search"}],
                model="gpt-4o",
            )

Ora che abbiamo creato il nostro asisstente e lo abbiamo dotato della "capacità" di leggere i files, dobbiamo associarlo ad un vectorstore.  
Per farlo utilizziamo il metodo update dell'assistente nell'sdk

```python
client.beta.assistants.update(
            assistant_id= # qua mettiamo l'id dell'assistente creato prima,
            tool_resources={"file_search": {"vector_store_ids": [vectorstore_id]}},
)

In [51]:
client.beta.assistants.update(
            assistant_id=new_assistant.id,
            tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

Assistant(id='asst_yuF9K7LCwWtnXqkWKiD0PLCF', created_at=1717577229, description=None, instructions='Sei un helper aziendale e devi aiutare a rispondere a delle domande basate sulla documentazione che ti è stata fornita.', metadata={}, model='gpt-4o', name='Assistente di prova', object='assistant', tools=[FileSearchTool(type='file_search')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_emFudN5h5cw92LnDnaNSf7Qv'])), top_p=1.0)

Ora, il nostro Assistant sarà dotato della conoscenza sul documento che gli abbiamo passato. Passiamo ora alla gestione delle conversazioni.

# Gestione delle conversazioni

## Creazione di un thread

Un thread è un oggetto di openai che funge da contenitore per i vari messaggi.  
 Quando diciamo a openai di crearne uno, lo facciamo istanziando un oggetto thread.  
 Questo oggetto (nell'esempio qua sotto chiamato "primo_thread") dopo l'esecuzione della riga, avrà un attributo primo_thread.id, che conterrà l'id del thread all'interno di openai.



In [52]:
primo_thread = client.beta.threads.create()
primo_thread.id

'thread_iqCMycpjy6POMapw1SrgfQsH'

## Messaggi

I messaggi sono l'unità fondamentale con cui funzionano gli assistants, e più in generale i modelli chat di AI. I messaggi possono avere ruoli diversi, ma quelli fondamentali sono due:
- User: Messaggio dell'utente.
- Assistant: Messagio dell'assistente. Quest messaggi vanno passati al modello quando l'interazione va oltre il primo scambio di domanda e risposta. In questo modo il modello saprà il contenuto delle sue risposte precedenti e sarà in grado di proseguire il discorso senza ripetersi.

Esistono anche altri ruoli (ad esempio "function") ma non ci interessano attualmente in quanto riguardano funzionalità molto specifiche e che non ci servono in queste sede.

Quando creo un messaggio con OpenAI Assistants, devo istanziare un oggetto message specificando anche il thread di cui farà parte il messaggio che stiamo creando.

In [53]:
client.beta.threads.messages.create(
                thread_id=primo_thread.id,                          # thread vuoto creato in precedenza
                role="user",                                        # specifichiamo il ruolo user SEMPRE
                content="Come avviene il monitoraggio degli output?"   # contenuto della domanda
            )

Message(id='msg_El1eBTOLXUNjfr4qacp7wuA5', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Come avviene il monitoraggio degli output?'), type='text')], created_at=1717577230, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_iqCMycpjy6POMapw1SrgfQsH')

Openai ci dà la possibilità di scaricare la lista dei messaggi con il comando list messages. Questo ci consente di avere una "fotografia" del thread in un certo istante.
L'oggetto che si scarica con il comando list message è un oggetto iterabile che racchiude tanti oggetti message.

Gli oggetti message hanno questo body: (https://platform.openai.com/docs/api-reference/messages/createMessage)

ATTENZIONE: se utilizziamo l'sdk di openai (come nei nostri esempi) non sarà un json fatto di chiavi e valori, ma le chiavi saranno attributi dell'oggetto message che abbiamo creato.
```json
{
  "id": "msg_abc123",
  "object": "thread.message",
  "created_at": 1713226573,
  "assistant_id": null,
  "thread_id": "thread_abc123",
  "run_id": null,
  "role": "user",
  "content": [
    {
      "type": "text",
      "text": {
        "value": "How does AI work? Explain it in simple terms.",
        "annotations": []
      }
    }
  ],
  "attachments": [],
  "metadata": {}
}

```

In [54]:
messages = client.beta.threads.messages.list(
    thread_id=primo_thread.id
  )
messages

SyncCursorPage[Message](data=[Message(id='msg_El1eBTOLXUNjfr4qacp7wuA5', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Come avviene il monitoraggio degli output?'), type='text')], created_at=1717577230, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_iqCMycpjy6POMapw1SrgfQsH')], object='list', first_id='msg_El1eBTOLXUNjfr4qacp7wuA5', last_id='msg_El1eBTOLXUNjfr4qacp7wuA5', has_more=False)

L'oggetto SyncCursorPage[message] è un po' complesso, ma è pur sempre un iterabile. Per questo motivo possiamo trasformarlo in una lista usando la hotkey di python list().  
In questo modo otterremo una lista di elementi Message (che hanno la forma che segue il payload sopra.)

In [55]:
messages_list = list(messages)
messages_list[0].content[0].text.value

'Come avviene il monitoraggio degli output?'

## Run!

Il run è l'operazione attraverso cui creiamo le risposte del nostro assistente.  
Per eseguire un run abbiamo bisogno di un thread e di un assistente che generi la risposta.  
Come si può intuire, và eseguito quando l'ultimo messaggio della message list del thread è di tipo user.

In [56]:
client.beta.threads.runs.create_and_poll(
                thread_id=primo_thread.id,          # id del thread creato in precedenza
                assistant_id=new_assistant.id       # id dell'assistente con file search creato in precedenza
)  

Run(id='run_iBCJIBXPR4JxzMmhuuENcJL0', assistant_id='asst_yuF9K7LCwWtnXqkWKiD0PLCF', cancelled_at=None, completed_at=1717577247, created_at=1717577231, expires_at=None, failed_at=None, incomplete_details=None, instructions='Sei un helper aziendale e devi aiutare a rispondere a delle domande basate sulla documentazione che ti è stata fornita.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=1717577232, status='completed', thread_id='thread_iqCMycpjy6POMapw1SrgfQsH', tool_choice='auto', tools=[FileSearchTool(type='file_search')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=511, prompt_tokens=16943, total_tokens=17454), temperature=1.0, top_p=1.0, tool_resources={})

Se scarichiamo nuovamente la lista dei messaggi, vedremo che c'è un nuovo elemento:

In [57]:
messages = client.beta.threads.messages.list(
    thread_id=primo_thread.id
  )

for m in messages:
    print(m)

Message(id='msg_t2ElSvMW8s9sCM4GzUAJ9dXe', assistant_id='asst_yuF9K7LCwWtnXqkWKiD0PLCF', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=358, file_citation=FileCitation(file_id='file-XTTie8G2pSLpFe5rm2zZwEKo', quote=None), start_index=346, text='【4:0†source】', type='file_citation'), FileCitationAnnotation(end_index=1396, file_citation=FileCitation(file_id='file-XTTie8G2pSLpFe5rm2zZwEKo', quote=None), start_index=1384, text='【4:0†source】', type='file_citation')], value="Il monitoraggio degli output avviene attraverso diverse pratiche di supervisione, strumenti tecnici e politiche formali, tra cui:\n\n1. **Strumenti di gestione dei log**: La società utilizza uno strumento di gestione dei log per identificare eventi che potrebbero influire sulla capacità della società di raggiungere i propri obiettivi di sicurezza【4:0†source】.\n\n2. **IDS (Intrusion Detection System)**: Utilizza un IDS per fornire un monitoraggio continu

Questo nuovo elemento sarà la nostra risposta:  
PS: con list_messages, i messaggi della lista sono ordinati dal più recente al meno recente

In [58]:
messages_list = list(messages)
messages_list[0].content[0].text.value

"Il monitoraggio degli output avviene attraverso diverse pratiche di supervisione, strumenti tecnici e politiche formali, tra cui:\n\n1. **Strumenti di gestione dei log**: La società utilizza uno strumento di gestione dei log per identificare eventi che potrebbero influire sulla capacità della società di raggiungere i propri obiettivi di sicurezza【4:0†source】.\n\n2. **IDS (Intrusion Detection System)**: Utilizza un IDS per fornire un monitoraggio continuo della rete aziendale e per la precoce individuazione di potenziali violazioni della sicurezza .\n\n3. **Politiche formali**: Le politiche formali della società delineano i requisiti per la gestione delle vulnerabilità e il monitoraggio del sistema .\n\n4. **Scanner di vulnerabilità**: Vengono eseguite scansioni continue delle vulnerabilità sui sistemi esterni. Le vulnerabilità critiche e di alto livello sono monitorate fino alla loro risoluzione .\n\n5. **Test di penetrazione**: Viene effettuato almeno annualmente un test di penetrazi

## Eliminazioni

Ricopre una grande importanza l'eliminazione delle risorse. Vediamo ora i vari comandi utilizzati per eliminare un thread, un assistente e un vectorstore.  
I response associati all'operazione ci danno conferma dell'esito positivo dell'eliminazione.  

In [59]:
# Per eliminare un thread uso il comando:
response_thread = client.beta.threads.delete(
  primo_thread.id
  )
print(response_thread)
# Per eliminare un Assistant uso il comando:
response_assistant = client.beta.assistants.delete(
  new_assistant.id
  )
print(response_assistant)
# Per eliminare un vS utilizzo il comando:
response_vs = client.beta.vector_stores.delete(
  vector_store_id=vector_store.id
)
print(response_vs)

ThreadDeleted(id='thread_iqCMycpjy6POMapw1SrgfQsH', deleted=True, object='thread.deleted')
AssistantDeleted(id='asst_yuF9K7LCwWtnXqkWKiD0PLCF', deleted=True, object='assistant.deleted')
VectorStoreDeleted(id='vs_emFudN5h5cw92LnDnaNSf7Qv', deleted=True, object='vector_store.deleted')
